## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Clinical Deidentification Romanian

Deidentification is essential for safeguarding patient privacy in clinical data, including texts, PDFs, images, and DICOM files containing Protected Health Information (PHI). PHI encompasses various health-related data, including common identifiers such as name, address, birth date, and Social Security Number.

- **Model**: `ro.deid.clinical`
- **Model Description**: This pipeline is trained with w2v_cc_300d Romanian embeddings and can be used to deidentify PHI information from medical texts in Romanian. The PHI information will be masked and obfuscated in the resulting text. The pipeline can mask, fake or obfuscate the following entities: AGE, CITY, COUNTRY, DATE, DOCTOR, EMAIL, FAX, HOSPITAL, IDNUM, LOCATION-OTHER, MEDICALRECORD, ORGANIZATION, PATIENT, PHONE, PROFESSION, STREET, ZIP, ACCOUNT, LICENSE, PLATE

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "ro-deid-clinical"

content_type = "application/json"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"


### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

---------!

Once endpoint has been created, you would be able to perform real-time inference.

In [6]:
import json
import pandas as pd
import os
import boto3


# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


def process_data_and_invoke_realtime_endpoint(data_dicts):
    for data_dict in data_dicts:
        json_input_data = json.dumps(data_dict, ensure_ascii=False)
        i = 1
        input_file_name = f'inputs/real-time/input{i}.json'
        output_file_name = f'outputs/real-time/out{i}.out'

        while os.path.exists(input_file_name) or os.path.exists(output_file_name):
            i += 1
            input_file_name = f'inputs/real-time/input{i}.json'
            output_file_name = f'outputs/real-time/out{i}.out'

        os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
        os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

        with open(input_file_name, 'w', encoding="utf-8") as f:
            f.write(json_input_data)

        s3_client.put_object(Bucket=s3_bucket, Key=f"{model_name}/validation-input-json/real-time/{os.path.basename(input_file_name)}", Body=bytes(json_input_data.encode('UTF-8')))

        response = sm_runtime.invoke_endpoint(
            EndpointName=model_name,
            ContentType=content_type,
            Accept="application/json",
            Body=json_input_data,
        )

        # Process response
        response_data = json.loads(response["Body"].read().decode("utf-8"))
        df = pd.DataFrame(response_data)
        display(df)

        # Save response data to file
        with open(output_file_name, 'w') as f_out:
            json.dump(response_data, f_out, indent=4 , ensure_ascii=False)

### Initial Setup

In [7]:
docs = [
'''LABORATOR RADStrada AbabeidaSacueni, 354573i, 3545730265-21011065-210110 ,OFFICE@SMURDICE@SMURD
Data setului de a10 May 2022 May 2022 18:45:00
InregistPAUL DANIELUTUDORIEL TUDOR 
Nume si pMILASAN DANUTSAN DANUT 
45rsta : 45 , Sex : Mascul456 45 789 2 45 789 2 
damut__d@hotmail.comtmail.com 1761218264378218264378 
Cod p627480543615010543615010
LK0004567SK0004567S, 
ÎnmatriBD32904: BD32904,SEWS324095192710408192710408,

LA ORA : 10/12/2021 14:54_ 
VALIDATIOANA GHIBANNA GHIBAN ; 
INVESTIGATII : CT TORACICÃ NATIVÃ''',

'''C.N.P : 2450Agota EVELYNata setului245050226440125 May 2022 
Varsta : 77,..25 May 2022Dr. Agota E77LYN, 
C.N.P : 245050BUREAN MARIAa setul+40(235)413773 25 May 2022maria@gmail.comNume si PrenB004256985MN MARIA 
Tel: +40(2CD2051133, 
E-mailFXHZ7170951927104999LicSpitalul Pentru Ochi, Drumul de Deal CNr. 972,Vaslui :737405170951927104999, 
Spitalul Pentru Ochi, Drumul de Deal ,Nr. 972 Vaslui, 737405

bolus 10 ml Iomeron 350 urmate de 30 ml flush salin cu 5 ml/s .
Se continua cu 65 ml contrast si 30 ml flush salin , cu acelasi flux .
Se efectueaza o examinare angio-CT coronariana cu achizitii spirale cu reconstructii retrospective la o frecventa cardiaca medie de 60/min .
Opacifiere buna a patului vascular si cavitatilor cardiace .
Fara incidente sau accidente pe parcursul examinarii .

CONCLUZII : - Stare dupa stentare ramus intermedius si artera coronara dreapta .
Aspect corespunzator al stenturilor , cu minima stenoza la nivelul ACD si cu flux distal corespunzator.'''

]


sample_text = """TOMOGRAFIA COMPUTERIZATA A CREIERULUI INVESTIGATII REZULTAT DLP
Medic Laborator : BURIAN MONICA
PROTOCOL DE EXPLORARE PRIN COMPUTER TOMOGRAFIE DATA : 11 FEBRUARIE 2022
Nume si prenume : RUS OLIMPIU DUMITRU , 49 ani, Sex : M
TEL : +40 67 5745,
E-mail : olimpiu_d@gmail.com
CNP : 1730104060763
Licență : E000198985A, Înmatriculare : CD32156, Cont : LZWZ7170951927104999,
Solicitare : Angio CT coronarian
kg : 100 Dg .
de trimitere Stare post-AVCI .
Stare post-trombectomie .
Proces inlocuitor de spatiu ventricular stang
Trimis de : Sectia Clinica Neurologie 1a SCJU Tirgu Mures
Medic : Dr.Melanie Dana
CONCLUZII : - Tromb ventricular stang de 20/40/17 mm , - Leziuni sechelar ischemice in segmentele apicale septal si inferior , cu aspect anevrismal apical VS - Leziune non-ischemica/MINOCA ?
48 mediocardiaca septala - Examenul AngioCT coronarian nu pune in evidenta ateromatoza sau stenoze coronariene semnificative.
"""

### Important Parameters

- **masking_policy**: `str`

    Users can select a masking policy to determine how sensitive entities are handled:

    Example: "**LABORATOR RADStrada AbabeidaSacueni, 354573i, 3545730265-21011065-210110 ,OFFICE@SMURDICE@SMURD**"

    - **masked**: Default policy that masks entities with their type.

      -> 'LABORATOR RADIOLOGIE, `<STREET>`, `<CITY>`, `<ZIP>` , TEL : `<PHONE>` , E-MAIL: `<EMAIL>`'

    - **obfuscated**: Replaces sensitive entities with random values of the same type.

      -> 'LABORATOR RADIOLOGIE, `Intrarea Diaconescu`, `Aiud`, `302784` , TEL : `0263 144 119` , E-MAIL: `jeneltudor@email.ro`'

    - **masked_fixed_length_chars**: Masks entities with a fixed length of asterisks (\*).

      -> 'LABORATOR RADIOLOGIE, `****`, `****`, `****` , TEL : `****` , E-MAIL: `****`'

    - **masked_with_chars**: Masks entities with asterisks (\*).

      -> 'LABORATOR RADIOLOGIE, [`***********`], [`*****`], [`****`] , TEL : [`*********`] , E-MAIL: [`**********`]'

- **sep**: `str`

    Separator used to join subparts within each prediction.

    By default, the separator is set to a single space (" "), but users can specify any other separator as needed. Necessary because the model outputs predictions as separate subparts, and the chosen separator is used to join them into coherent text.

    The separator must be one of the following characters: space (' '), newline ('\n'), comma (','), tab ('\t'), or colon (':').
    
You can specify these parameters in the input as follows:

```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ],
    "masking_policy": "masked",
    "sep": " ",
}
```

### **Input format**: Single Text Document

Provide a single text document as a string.

  
  
```json
{
    "text": "Single text document"
}
```

In [8]:
# masked (default-policy)
data_dicts = [
    {
        "text": sample_text
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,predictions
0,"TOMOGRAFIA COMPUTERIZATA A CREIERULUI INVESTIGATII REZULTAT DLP Medic Laborator : <DOCTOR> PROTOCOL DE EXPLORARE PRIN COMPUTER TOMOGRAFIE DATA : <DATE> Nume si prenume : <PATIENT> , <AGE> ani, Sex : M\nTEL : <PHONE>,\nE-mail : <EMAIL> CNP : <IDNUM> Licență : <LICENSE>, Înmatriculare : <PLATE>, Cont : <ACCOUNT>,\nSolicitare : Angio CT coronarian\nkg : 100 Dg .\nde trimitere Stare post-AVCI . Stare post-trombectomie . Proces inlocuitor de spatiu ventricular stang Trimis de : Sectia Clinica Neurologie 1a <HOSPITAL> Medic : <DOCTOR> CONCLUZII : - Tromb ventricular stang de 20/40/17 mm , - Leziuni sechelar ischemice in segmentele apicale septal si inferior , cu aspect anevrismal apical VS - Leziune non-ischemica/MINOCA ? 48 mediocardiaca septala - Examenul AngioCT coronarian nu pune in evidenta ateromatoza sau stenoze coronariene semnificative."


In [9]:
# obfuscated
data_dicts = [
    {
        "text": sample_text,
        "masking_policy": "obfuscated"
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,predictions
0,"TOMOGRAFIA COMPUTERIZATA A CREIERULUI INVESTIGATII REZULTAT DLP Medic Laborator : Doina Gheorghiu PROTOCOL DE EXPLORARE PRIN COMPUTER TOMOGRAFIE DATA : 11 FEBRUARIE 2022 Nume si prenume : Dinut Liliana Daniela , 83 ani, Sex : M\nTEL : +16 10 9604,\nE-mail : lstancu@zzup.ro CNP : 5409811914782 Licență : N562130865H, Înmatriculare : QI69629, Cont : SEAE8282288015490352,\nSolicitare : Angio CT coronarian\nkg : 100 Dg .\nde trimitere Stare post-AVCI . Stare post-trombectomie . Proces inlocuitor de spatiu ventricular stang Trimis de : Sectia Clinica Neurologie 1a Institutul Clinic de Urologie si Transplant Renal Medic : Agata Voinea CONCLUZII : - Tromb ventricular stang de 20/40/17 mm , - Leziuni sechelar ischemice in segmentele apicale septal si inferior , cu aspect anevrismal apical VS - Leziune non-ischemica/MINOCA ? 48 mediocardiaca septala - Examenul AngioCT coronarian nu pune in evidenta ateromatoza sau stenoze coronariene semnificative."


### **Input format**: Array of Text Documents

Use an array containing multiple text documents. Each element represents a separate text document.

```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ]
}
```

In [10]:
# masked (default-policy)
data_dicts = [
    {
        "text": docs
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,predictions
0,"LABORATOR RADStrada AbabeidaSacueni, <ZIP>, <ZIP> ,<EMAIL> Data setului de <DATE> 18:45:00 InregistPAUL <ORGANIZATION> Nume si <ORGANIZATION> 45rsta : 45 , Sex : Mascul456 45 789 2 45 789 2 \n<EMAIL> <IDNUM> Cod <IDNUM> <IDNUM>, ÎnmatriBD32904: BD32904,SEWS324095192710408192710408,\n\nLA ORA : <DATE> 14:54_ VALIDATIOANA GHIBANNA GHIBAN ; \nINVESTIGATII : CT TORACICÃ NATIVÃ"
1,"C.N.P : 2450Agota EVELYNata <IDNUM> Varsta : <AGE> May 2022Dr. <DOCTOR>, \nC.N.P : 245050BUREAN MARIAa <PHONE> <DATE> si <IDNUM> <DOCTOR> Tel: <PHONE>, \nE-mailFXHZ7170951927104999LicSpitalul Pentru Ochi, <STREET> de Deal CNr. 972,Vaslui :<IDNUM>, \n<HOSPITAL>, <STREET> ,<STREET> <CITY>, <ZIP>\n\nbolus 10 ml Iomeron 350 urmate de 30 ml flush salin cu 5 ml/s . Se continua cu 65 ml contrast si 30 ml flush salin , cu acelasi flux . Se efectueaza o examinare angio-CT coronariana cu achizitii spirale cu reconstructii retrospective la o frecventa cardiaca medie de 60/min . Opacifiere buna a patului vascular si cavitatilor cardiace . Fara incidente sau accidente pe parcursul examinarii . CONCLUZII : - Stare dupa stentare ramus intermedius si artera coronara dreapta . Aspect corespunzator al stenturilor , cu minima stenoza la nivelul ACD si cu flux distal corespunzator."


In [11]:
# obfuscated
data_dicts = [
    {
        "text": docs,
        "masking_policy": "obfuscated"
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,predictions
0,"LABORATOR RADStrada AbabeidaSacueni, 161096E, 4540981191-47829562-130865 ,tudorsmaranda@kappa.ro Data setului de a10 May 2022 May 2022 18:45:00 InregistPAUL STAR FALEZA HOUSE SRL Nume si DRYNET TRADING SRL 45rsta : 45 , Sex : Mascul456 45 789 2 45 789 2 \nsabin66@zzup.ro 7846962952841324401027 Cod O536644034742595638756433 IR5188416SA6301601U, ÎnmatriBD32904: BD32904,SEWS324095192710408192710408,\n\nLA ORA : 10/12/2021 14:54_ VALIDATIOANA GHIBANNA GHIBAN ; \nINVESTIGATII : CT TORACICÃ NATIVÃ"
1,"C.N.P : 2450Agota EVELYNata XNATFTD322025427062376 EGB 1517 Varsta : 94 May 2022Dr. NISTOR Eliana, \nC.N.P : 245050BUREAN MARIAa OHYWV+37(106)269485 25 May 2022maria@gmail.comNume si IOEVO350093818EX Doina Gheorghiu Tel: +93(7JI9678938, \nE-mailFXHZ7170951927104999LicSpitalul Pentru Ochi, Bulevardul Cristea de Deal CNr. 972,Vaslui :101751025852778242353, \nAcademia Româna Spitalul Universitar de Urgenta “Elias”, Aleea Nistor ,Intrarea Diaconescu Curcani, 614431\n\nbolus 10 ml Iomeron 350 urmate de 30 ml flush salin cu 5 ml/s . Se continua cu 65 ml contrast si 30 ml flush salin , cu acelasi flux . Se efectueaza o examinare angio-CT coronariana cu achizitii spirale cu reconstructii retrospective la o frecventa cardiaca medie de 60/min . Opacifiere buna a patului vascular si cavitatilor cardiace . Fara incidente sau accidente pe parcursul examinarii . CONCLUZII : - Stare dupa stentare ramus intermedius si artera coronara dreapta . Aspect corespunzator al stenturilor , cu minima stenoza la nivelul ACD si cu flux distal corespunzator."


### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [12]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [13]:
import os

validation_file_name_1 = "input_1.json"
validation_file_name_2 = "input_2.json"
validation_file_name_3 = "input_3.json"
validation_file_name_4 = "input_4.json"

validation_input_path = f"s3://{s3_bucket}/{model_name}/validation-input-json/batch"
validation_output_path = f"s3://{s3_bucket}/{model_name}/validation-output-json/batch"

input_dir = 'inputs/batch'
output_dir = 'outputs/batch'

os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

In [14]:
import json

def write_and_upload_to_s3(json_input_data, file_name):

    json_data = json.dumps(json_input_data, ensure_ascii=False)

    with open(file_name, "w", encoding="utf-8") as f:
        f.write(json_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input-json/batch/{os.path.basename(file_name)}",
        Body=(bytes(json_data.encode("UTF-8"))),
    )

In [15]:
# Define input JSON data for each validation file
input_json_data = {
    validation_file_name_1: {"text": docs},
    validation_file_name_2: {"text": docs, "masking_policy": "obfuscated"},
    validation_file_name_3: {"text": docs, "masking_policy": "masked_fixed_length_chars"},
    validation_file_name_4: {"text": docs, "masking_policy": "masked_with_chars"},
}

# Write and upload each input JSON data to S3
for file_name, json_data in input_json_data.items():
    write_and_upload_to_s3(json_data, f"{input_dir}/{file_name}")

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type= batch_transform_inference_instance_type,
    accept="application/json",
)
transformer.transform(validation_input_path, content_type=content_type)
transformer.wait()

In [17]:
from urllib.parse import urlparse

def process_s3_output_and_save(validation_file_name, output_file_name):

    output_file_path = f"{output_dir}/{output_file_name}"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}/{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data)
    display(df)

    # Save the data to the output file
    with open(output_file_path, 'w', encoding='utf-8') as f_out:
        json.dump(data, f_out, indent=4, ensure_ascii=False)

#### masked (default-policy)

In [18]:
process_s3_output_and_save(validation_file_name_1, "out_1.out")

,predictions
0,"LABORATOR RADStrada AbabeidaSacueni, <ZIP>, <ZIP> ,<EMAIL> Data setului de <DATE> 18:45:00 InregistPAUL <ORGANIZATION> Nume si <ORGANIZATION> 45rsta : 45 , Sex : Mascul456 45 789 2 45 789 2 \n<EMAIL> <IDNUM> Cod <IDNUM> <IDNUM>, ÎnmatriBD32904: BD32904,SEWS324095192710408192710408,\n\nLA ORA : <DATE> 14:54_ VALIDATIOANA GHIBANNA GHIBAN ; \nINVESTIGATII : CT TORACICÃ NATIVÃ"
1,"C.N.P : 2450Agota EVELYNata <IDNUM> Varsta : <AGE> May 2022Dr. <DOCTOR>, \nC.N.P : 245050BUREAN MARIAa <PHONE> <DATE> si <IDNUM> <DOCTOR> Tel: <PHONE>, \nE-mailFXHZ7170951927104999LicSpitalul Pentru Ochi, <STREET> de Deal CNr. 972,Vaslui :<IDNUM>, \n<HOSPITAL>, <STREET> ,<STREET> <CITY>, <ZIP>\n\nbolus 10 ml Iomeron 350 urmate de 30 ml flush salin cu 5 ml/s . Se continua cu 65 ml contrast si 30 ml flush salin , cu acelasi flux . Se efectueaza o examinare angio-CT coronariana cu achizitii spirale cu reconstructii retrospective la o frecventa cardiaca medie de 60/min . Opacifiere buna a patului vascular si cavitatilor cardiace . Fara incidente sau accidente pe parcursul examinarii . CONCLUZII : - Stare dupa stentare ramus intermedius si artera coronara dreapta . Aspect corespunzator al stenturilor , cu minima stenoza la nivelul ACD si cu flux distal corespunzator."


#### obfuscated

In [19]:
process_s3_output_and_save(validation_file_name_2, "out_2.out")

,predictions
0,"LABORATOR RADStrada AbabeidaSacueni, 161096E, 4540981191-47829562-130865 ,tudorsmaranda@kappa.ro Data setului de a10 May 2022 May 2022 18:45:00 InregistPAUL STAR FALEZA HOUSE SRL Nume si DRYNET TRADING SRL 45rsta : 45 , Sex : Mascul456 45 789 2 45 789 2 \nsabin66@zzup.ro 7846962952841324401027 Cod O536644034742595638756433 IR5188416SA6301601U, ÎnmatriBD32904: BD32904,SEWS324095192710408192710408,\n\nLA ORA : 10/12/2021 14:54_ VALIDATIOANA GHIBANNA GHIBAN ; \nINVESTIGATII : CT TORACICÃ NATIVÃ"
1,"C.N.P : 2450Agota EVELYNata ZOXWRUE454098119147829 FAO 1308 Varsta : 94 May 2022Dr. NISTOR Eliana, \nC.N.P : 245050BUREAN MARIAa MVHQI+69(629)528413 25 May 2022maria@gmail.comNume si KGMWN027253664QI Doina Gheorghiu Tel: +34(7QQ5956387, \nE-mailFXHZ7170951927104999LicSpitalul Pentru Ochi, Bulevardul Cristea de Deal CNr. 972,Vaslui :564332951884166063016, \nAcademia Româna Spitalul Universitar de Urgenta “Elias”, Aleea Nistor ,Intrarea Diaconescu Curcani, 010932\n\nbolus 10 ml Iomeron 350 urmate de 30 ml flush salin cu 5 ml/s . Se continua cu 65 ml contrast si 30 ml flush salin , cu acelasi flux . Se efectueaza o examinare angio-CT coronariana cu achizitii spirale cu reconstructii retrospective la o frecventa cardiaca medie de 60/min . Opacifiere buna a patului vascular si cavitatilor cardiace . Fara incidente sau accidente pe parcursul examinarii . CONCLUZII : - Stare dupa stentare ramus intermedius si artera coronara dreapta . Aspect corespunzator al stenturilor , cu minima stenoza la nivelul ACD si cu flux distal corespunzator."


#### masked_fixed_length_chars

In [20]:
process_s3_output_and_save(validation_file_name_3, "out_3.out")

,predictions
0,"LABORATOR RADStrada AbabeidaSacueni, ****, **** ,**** Data setului de **** 18:45:00 InregistPAUL **** Nume si **** 45rsta : 45 , Sex : Mascul456 45 789 2 45 789 2 \n**** **** Cod **** ****, ÎnmatriBD32904: BD32904,SEWS324095192710408192710408,\n\nLA ORA : **** 14:54_ VALIDATIOANA GHIBANNA GHIBAN ; \nINVESTIGATII : CT TORACICÃ NATIVÃ"
1,"C.N.P : 2450Agota EVELYNata **** Varsta : **** May 2022Dr. ****, \nC.N.P : 245050BUREAN MARIAa **** **** si **** **** Tel: ****, \nE-mailFXHZ7170951927104999LicSpitalul Pentru Ochi, **** de Deal CNr. 972,Vaslui :****, \n****, **** ,**** ****, ****\n\nbolus 10 ml Iomeron 350 urmate de 30 ml flush salin cu 5 ml/s . Se continua cu 65 ml contrast si 30 ml flush salin , cu acelasi flux . Se efectueaza o examinare angio-CT coronariana cu achizitii spirale cu reconstructii retrospective la o frecventa cardiaca medie de 60/min . Opacifiere buna a patului vascular si cavitatilor cardiace . Fara incidente sau accidente pe parcursul examinarii . CONCLUZII : - Stare dupa stentare ramus intermedius si artera coronara dreapta . Aspect corespunzator al stenturilor , cu minima stenoza la nivelul ACD si cu flux distal corespunzator."


#### masked_with_chars

In [21]:
process_s3_output_and_save(validation_file_name_4, "out_4.out")

,predictions
0,"LABORATOR RADStrada AbabeidaSacueni, [*****], [************************] ,[*******************] Data setului de [*******************] 18:45:00 InregistPAUL [*******************] Nume si [*********************] 45rsta : 45 , Sex : Mascul456 45 789 2 45 789 2 \n[***************************] [********************] Cod [***********************] [*****************], ÎnmatriBD32904: BD32904,SEWS324095192710408192710408,\n\nLA ORA : [********] 14:54_ VALIDATIOANA GHIBANNA GHIBAN ; \nINVESTIGATII : CT TORACICÃ NATIVÃ"
1,"C.N.P : 2450Agota EVELYNata [*****************************] Varsta : [*****] May 2022Dr. [**********], \nC.N.P : 245050BUREAN MARIAa [*****************] [****************************] si [**************] [***] Tel: [************], \nE-mailFXHZ7170951927104999LicSpitalul Pentru Ochi, [****] de Deal CNr. 972,Vaslui :[*******************], \n[******************], [************] ,[*****] [****], [****]\n\nbolus 10 ml Iomeron 350 urmate de 30 ml flush salin cu 5 ml/s . Se continua cu 65 ml contrast si 30 ml flush salin , cu acelasi flux . Se efectueaza o examinare angio-CT coronariana cu achizitii spirale cu reconstructii retrospective la o frecventa cardiaca medie de 60/min . Opacifiere buna a patului vascular si cavitatilor cardiace . Fara incidente sau accidente pe parcursul examinarii . CONCLUZII : - Stare dupa stentare ramus intermedius si artera coronara dreapta . Aspect corespunzator al stenturilor , cu minima stenoza la nivelul ACD si cu flux distal corespunzator."


In [22]:
model.delete_model()

INFO:sagemaker:Deleting model with name: ro-deid-clinical-2024-04-02-18-52-43-145


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

